In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

## Take a quick look at some [HDX food insecurity data on Syria](https://data.humdata.org/dataset/syrian-arab-republic-food-security-agriculture-severity-classification#)

In [13]:
df = pd.read_excel('data/external/syr-food-sector-pin_severity_2024.xlsx', sheet_name='Food_PiN_by_severity_HNO_2024', header=1)

In [14]:
df.head()

,#adm1 +name,#adm1 +code,#adm2 +name,#adm2 +code,#adm3 +name,#adm3 +code,#Indicator +num,#inneed +ind
0,Damascus,SY01,Damascus,SY0100,Damascus,SY010000,3,898967.999742
1,Aleppo,SY02,Ain Al Arab,SY0206,Lower Shyookh,SY020601,3,12597.570000
2,Aleppo,SY02,Al Bab,SY0202,Dayr Hafir,SY020202,3,20229.982609
3,Aleppo,SY02,Ain Al Arab,SY0206,Ain al Arab,SY020600,3,60936.990000
4,Aleppo,SY02,A'zaz,SY0204,Tall Refaat,SY020402,3,4928.040000


In [15]:
with open("../data/processed/en_locations_reversed.pkl", "rb") as f:
    locations_dict_en_reversed = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/en_locations_reversed.pkl'

In [ ]:
get_locations_from_text(test_article,locations_lookup=locations_dict_en_reversed, locations_pattern=locations_pattern_en)